# グレーコード

グレーコードとは。。。

In [1]:
from z3 import *

以下はz3でNビットのグレーコードを生成するプログラムです。連続二つのコードの差分の絶対値は2の乗数という制約条件。。。。

In [36]:
def gray_code(nbits):
    n = 2 ** nbits
    codes = IntVector('code', n)
    solver = Solver()
    
    for c in codes:
        solver.add(0 <= c, c < n)
        
    solver.add(Distinct(codes))
    
    for c1, c2 in zip(codes, codes[1:] + codes[:1]):
        diff = Abs(c1 - c2)
        solver.add(Or([diff == 2**bit for bit in range(nbits)]))
    
    solver.add(codes[0] == 0)
    solver.check()
    model = solver.model()
    return [model[c].as_long() for c in codes]

nbits = 5
%time codes = gray_code(nbits)
for c in codes:
    print(f'{c:0{nbits}b}')

CPU times: total: 3.23 s
Wall time: 3.36 s
00000
01000
11000
11010
11100
10100
10000
10001
11001
10101
00101
01001
00001
00011
10011
11011
10111
01111
00111
01011
01101
11101
11111
11110
01110
00110
10110
10010
00010
01010
01100
00100


上のコードの実行速度は遅くて、次ぐは`Int`のかわりに`BitVec`を使います。前後二つコードの差分はXOR演算子(^)で計算し、差分は一つのビットが1しかないを判定するには、次の方法を使います。

`d & (d - 1) = 0, d != 0`

In [47]:
def gray_code_bv(nbits):
    max_value = 2 ** nbits - 1
    codes = [BitVec(f'code_{i}', nbits) for i in range(2**nbits)]
    solver = Solver()
    solver.add(Distinct(codes))
    
    for c1, c2 in zip(codes, codes[1:] + codes[:1]):
        diff = c1 ^ c2
        # solver.add(Or([diff == 2**bit for bit in range(nbits)]))
        solver.add(diff & (diff - 1) == 0, diff != 0)
    
    solver.add(codes[0] == 0)
    
    solver.check()
    model = solver.model()
    return [model[c].as_long() for c in codes]

これで速度は数十倍速くなります。

In [53]:
%time codes = gray_code_bv(nbits)
for c in codes:
    print(f'{c:0{nbits}b}')

CPU times: total: 62.5 ms
Wall time: 64.3 ms
00000
01000
11000
11001
01001
01101
00101
00100
01100
11100
11110
01110
01010
11010
10010
00010
00110
10110
10111
00111
01111
01011
00011
10011
11011
11111
11101
10101
10100
10000
10001
00001
